In [2]:
!pip3 install beautifulsoup4
!pip3 install requests

In [3]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [4]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name

In [5]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [6]:
# TASK 1: Request the page and create BeautifulSoup object
response = requests.get(static_url)
soup = BeautifulSoup(response.content, 'html.parser')
print(soup.title.string)

List of Falcon 9 and Falcon Heavy launches - Wikipedia


In [7]:
# TASK 2: Extract column names
html_tables = soup.find_all('table', class_='wikitable')

In [8]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a><sup class="reference" id="cite_ref-booster_11-2"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-2"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9

In [9]:
# Extract column names from table headers
column_names = []
for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)
    if name and len(name) > 0:
        column_names.append(name)
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [12]:
# TASK 3: Create launch_dict and parse tables
launch_dict = dict.fromkeys(column_names)
del launch_dict['Date and time ( )']  # Remove irrelevant column

# Initialize dictionary with empty lists
for key in launch_dict:
    launch_dict[key] = []
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

# Parse each table row
extracted_row = 0
for table in soup.find_all('table', class_='wikitable plainrowheaders collapsible'):
    for row in table.find_all('tr'):
        th = row.th
        if th and th.string and th.string.strip().isdigit():
            flight_number = th.string.strip()
            cells = row.find_all('td')
            if len(cells) < 9:  # Skip rows without enough data
                continue
            # Extract data using helper functions
            datatimelist = date_time(cells[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]
            bv = booster_version(cells[1])
            if not bv:
                bv = cells[1].a.string if cells[1].a else ""
            launch_site = cells[2].a.string if cells[2].a else ""
            payload = cells[3].a.string if cells[3].a else ""
            payload_mass = get_mass(cells[4])
            orbit = cells[5].a.string if cells[5].a else ""
            customer = cells[6].a.string if cells[6].a else ""
            launch_outcome = list(cells[7].strings)[0]
            booster_landing = landing_status(cells[8])
            # Append data to launch_dict
            launch_dict['Flight No.'].append(flight_number)
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)
            launch_dict['Version Booster'].append(bv)
            launch_dict['Launch site'].append(launch_site)
            launch_dict['Payload'].append(payload)
            launch_dict['Payload mass'].append(payload_mass)
            launch_dict['Orbit'].append(orbit)
            launch_dict['Customer'].append(customer)
            launch_dict['Launch outcome'].append(launch_outcome)
            launch_dict['Booster landing'].append(booster_landing)
            extracted_row += 1

# Create DataFrame and save as CSV
df = pd.DataFrame(launch_dict)
df.to_csv('spacex_web_scraped.csv', index=False)
print(f"Extracted {extracted_row} rows.")

Extracted 121 rows.
